# Fairness under Correlation Shifts

### This Jupyter Notebook simulates the proposed pre-processing approach on the synthetic data.

We consider two scenarios: supporting (1) a single metric (DP) and (2) multiple metrics (DP & EO).

We use FairBatch [Roh et al., ICLR 2021] as an in-processing approach.

## Import libraries

In [1]:
import sys, os
import numpy as np
import math
import random 
import itertools

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import torch

from models import LogisticRegression, weights_init_normal
from FairBatchSampler_Multiple import FairBatch, CustomDataset
from utils import correlation_reweighting, datasampling, test_model

import cvxopt
import cvxpy as cp
from cvxpy import OPTIMAL, Minimize, Problem, Variable, quad_form # Work in YJ kernel

import warnings
warnings.filterwarnings("ignore")

## Load and process the data

In the synthetic_data directory, there are a total of 6 numpy files including training data and test data.

In [2]:
xz_train = np.load('./synthetic_data/xz_train.npy')
y_train = np.load('./synthetic_data/y_train.npy') 
z_train = np.load('./synthetic_data/z_train.npy')

xz_test = np.load('./synthetic_data/xz_test.npy')
y_test = np.load('./synthetic_data/y_test.npy') 
z_test = np.load('./synthetic_data/z_test.npy')

xz_train = torch.FloatTensor(xz_train)
y_train = torch.FloatTensor(y_train)
z_train = torch.FloatTensor(z_train)

xz_test = torch.FloatTensor(xz_test)
y_test = torch.FloatTensor(y_test)
z_test = torch.FloatTensor(z_test)

## Hyperparameters and functions

In [3]:
seeds = [0,1,2,3,4]

w = np.array([sum((z_train==1)&(y_train==1))/len(y_train), sum((z_train==0)&(y_train==1))/len(y_train), sum((z_train==1)&(y_train==-1))/len(y_train), sum((z_train==0)&(y_train==-1))/len(y_train)])
corr = 0.18
alpha = 0.005 # Used in FairBatch

In [4]:
def run_epoch(model, train_features, labels, optimizer, criterion):
    """Trains the model with the given train data.

    Args:
        model: A torch model to train.
        train_features: A torch tensor indicating the train features.
        labels: A torch tensor indicating the true labels.
        optimizer: A torch optimizer.
        criterion: A torch criterion.

    Returns:
        loss value.
    """
    
    optimizer.zero_grad()

    label_predicted = model.forward(train_features)
    loss  = criterion((F.tanh(label_predicted.squeeze())+1)/2, (labels.squeeze()+1)/2)
    loss.backward()

    optimizer.step()
    
    return loss.item()

In [5]:
def find_w_cvxpy(w, corr, gamma1, gamma2):
    
    """Solves the SDP relaxation problem.

    Args:
        w: A list indicating the original data ratio for each (y, z)-class.
        corr: A real number indicating the target correlation.
        gamma1: A real number indicating the range of Pr(y) change
        gamma2: A real number indicating the range of Pr(z) change

    Returns:
        solution for the optimization problem.
    """
    
    n = len(w)
    a = w[0]
    b = w[1]
    c = w[2]
    d = w[3]
    orig_corr = w[0]/(w[0]+w[2]) - w[1]/(w[1]+w[3])

    P0 = np.array([[1,0,0,0,-a],[0,1,0,0,-b],[0,0,1,0,-c],[0,0,0,1,-d],[-a,-b,-c,-d,0]])
    
    P1 = np.array([[0,-corr/2,0,(1-corr)/2,0],[-corr/2,0,(-1-corr)/2,0,0],[0,(-1-corr)/2,0,-corr/2,0],[(1-corr)/2,0,-corr/2,0,0],[0,0,0,0,0]])

    P2 = np.array([[0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,0],[0,0,0,0,0],[1,1,0,0,0]])
    r2 = -2*(a+b)

    P3 = np.array([[0,0,0,0,1],[0,0,0,0,0],[0,0,0,0,1],[0,0,0,0,0],[1,0,1,0,0]])
    r3 = -2*(a+c)

    P4 = np.array([[0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,1],[1,1,1,1,0]])
    r4 = -2*1

    P5 = np.array([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,1]])

    X = cp.Variable((n+1,n+1), symmetric=True)

    constraints = [X >> 0]
    constraints = [
        cp.trace(P1 @ X) == 0,
        cp.trace(P2 @ X) + r2 <= gamma1,
        cp.trace(P2 @ X) + r2 >= -gamma1,
        cp.trace(P3 @ X) + r3 <= gamma2,
        cp.trace(P3 @ X) + r3 >= -gamma2,
        cp.trace(P4 @ X) + r4 == 0,
        cp.trace(P5 @ X) == 1,
        X >> 0
    ]
    prob = cp.Problem(cp.Minimize(cp.trace(P0 @ X)),constraints)

    result = prob.solve()

    x = X.value
    x = x[:, -1][:-1]
    return x

# 1. Supporting a single metric (DP)
### The results are in the experiments of the paper.

## 1.1 In-processing-only (FairBatch)

In [6]:
train_type = 'in-processing-only'

full_tests = []
full_trains = []

""" Find new data ratio for each (y, z)-class """  
w_new = find_w_cvxpy(w, corr, 0.1, 0.1)

""" Find example weights according to the new weight """  
our_weights = correlation_reweighting(xz_train, y_train, z_train, w, w_new)

""" Train models """
for seed in seeds:

    print("< Seed: {} >".format(seed))

    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------

    useCuda = False
    if useCuda:
        model = LogisticRegression(3,1).cuda()
    else:
        model = LogisticRegression(3,1)

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    
    # ---------------------
    #  Set data and batch sampler
    # ---------------------
    
    if train_type == 'in-processing-only':
        train_data = CustomDataset(xz_train, y_train, z_train)
    else:
        new_index = datasampling(xz_train, y_train, z_train, our_weights, seed = seed)
        train_data = CustomDataset(xz_train[new_index], y_train[new_index], z_train[new_index])

    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = alpha, target_fairness = 'dp', replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    
    # ---------------------
    #  Model training
    # ---------------------

    for epoch in range(500):

        tmp_loss = []

        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)

        losses.append(sum(tmp_loss)/len(tmp_loss))
    
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, Unfairness (DP): {}".format(tmp_test['Acc'], tmp_test['DP_diff']))
    print("----------------------------------------------------------------------")
    

< Seed: 0 >
  Test accuracy: 0.8209999799728394, Unfairness (DP): 0.048428571428571376
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.8209999799728394, Unfairness (DP): 0.048428571428571376
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.8209999799728394, Unfairness (DP): 0.048428571428571376
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.8199999928474426, Unfairness (DP): 0.047428571428571376
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.8209999799728394, Unfairness (DP): 0.048428571428571376
----------------------------------------------------------------------


In [7]:
tmp_acc = []
tmp_unfair = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_unfair.append(full_tests[i]['DP_diff'])

print("Test accuracy (avg): {}".format(np.mean(tmp_acc)))
print("Unfairness (DP) (avg): {}".format(np.mean(tmp_unfair)))

Test accuracy (avg): 0.82079998254776
Unfairness (DP) (avg): 0.04822857142857138


## 1.2 Ours + In-processing (FairBatch)

In [8]:
train_type = 'ours'

full_tests = []
full_trains = []

""" Find new data ratio for each (y, z)-class """  
w_new = find_w_cvxpy(w, corr, 0.1, 0.1)

""" Find example weights according to the new weight """  
our_weights = correlation_reweighting(xz_train, y_train, z_train, w, w_new)

""" Train models """
for seed in seeds:

    print("< Seed: {} >".format(seed))

    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------

    useCuda = False
    if useCuda:
        model = LogisticRegression(3,1).cuda()
    else:
        model = LogisticRegression(3,1)

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    
    # ---------------------
    #  Set data and batch sampler
    # ---------------------
    
    if train_type == 'in-processing-only':
        train_data = CustomDataset(xz_train, y_train, z_train)
    else:
        new_index = datasampling(xz_train, y_train, z_train, our_weights, seed = seed)
        train_data = CustomDataset(xz_train[new_index], y_train[new_index], z_train[new_index])

    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = alpha, target_fairness = 'dp', replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    
    # ---------------------
    #  Model training
    # ---------------------

    for epoch in range(500):

        tmp_loss = []

        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)

        losses.append(sum(tmp_loss)/len(tmp_loss))
    
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, Unfairness (DP): {}".format(tmp_test['Acc'], tmp_test['DP_diff']))
    print("----------------------------------------------------------------------")
    

< Seed: 0 >
  Test accuracy: 0.8360000252723694, Unfairness (DP): 0.0034285714285714475
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.8349999785423279, Unfairness (DP): 0.0024285714285714466
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.8370000123977661, Unfairness (DP): 0.0008095238095238155
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.8360000252723694, Unfairness (DP): 0.0034285714285714475
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.8360000252723694, Unfairness (DP): 0.004190476190476189
----------------------------------------------------------------------


In [9]:
tmp_acc = []
tmp_unfair = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_unfair.append(full_tests[i]['DP_diff'])

print("Test accuracy (avg): {}".format(np.mean(tmp_acc)))
print("Unfairness (DP) (avg): {}".format(np.mean(tmp_unfair)))

Test accuracy (avg): 0.8360000133514405
Unfairness (DP) (avg): 0.0028571428571428693


# 2. Supporting multiple metrics (DP & EO)
### The results are in the experiments of the paper.

## 2.1 In-processing-only (FairBatch)

In [10]:
train_type = 'in-processing-only'

full_tests = []
full_trains = []

""" Find new data ratio for each (y, z)-class """  
w_new = find_w_cvxpy(w, corr, 0.1, 0.1)

""" Find example weights according to the new weight """  
our_weights = correlation_reweighting(xz_train, y_train, z_train, w, w_new)

""" Train models """
for seed in seeds:

    print("< Seed: {} >".format(seed))

    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------

    useCuda = False
    if useCuda:
        model = LogisticRegression(3,1).cuda()
    else:
        model = LogisticRegression(3,1)

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    
    # ---------------------
    #  Set data and batch sampler
    # ---------------------

    if train_type == 'in-processing-only':
        train_data = CustomDataset(xz_train, y_train, z_train)
    else:
        new_index = datasampling(xz_train, y_train, z_train, our_weights, seed = seed)
        train_data = CustomDataset(xz_train[new_index], y_train[new_index], z_train[new_index])
    
    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = alpha, target_fairness = 'dpeo', knob = 3, replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    
    # ---------------------
    #  Model training
    # ---------------------

    for epoch in range(500):

        tmp_loss = []

        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)

        losses.append(sum(tmp_loss)/len(tmp_loss))
    
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, Unfairness (DP & EO): {}".format(tmp_test['Acc'], max(tmp_test['DP_diff'], tmp_test['EqOdds_diff'])))
    print("----------------------------------------------------------------------")
    

< Seed: 0 >
  Test accuracy: 0.8479999899864197, Unfairness (DP & EO): 0.09999999999999999
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.8500000238418579, Unfairness (DP & EO): 0.08823529411764706
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.8489999771118164, Unfairness (DP & EO): 0.09215686274509803
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.8479999899864197, Unfairness (DP & EO): 0.08823529411764706
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.8479999899864197, Unfairness (DP & EO): 0.08823529411764706
----------------------------------------------------------------------


In [11]:
tmp_acc = []
tmp_unfair = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_unfair.append(max(full_tests[i]['DP_diff'], full_tests[i]['EqOdds_diff']))

print("Test accuracy (avg): {}".format(np.mean(tmp_acc)))
print("Unfairness (DP & EO) (avg): {}".format(np.mean(tmp_unfair)))

Test accuracy (avg): 0.8485999941825867
Unfairness (DP & EO) (avg): 0.09137254901960785


## 2.2 Ours + In-processing-only (FairBatch)

In [12]:
train_type = 'ours'

full_tests = []
full_trains = []

""" Find new data ratio for each (y, z)-class """  
w_new = find_w_cvxpy(w, corr, 0.1, 0.1)

""" Find example weights according to the new weight """  
our_weights = correlation_reweighting(xz_train, y_train, z_train, w, w_new)

""" Train models """
for seed in seeds:

    print("< Seed: {} >".format(seed))

    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------

    useCuda = False
    if useCuda:
        model = LogisticRegression(3,1).cuda()
    else:
        model = LogisticRegression(3,1)

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    
    # ---------------------
    #  Set data and batch sampler
    # ---------------------

    if train_type == 'in-processing-only':
        train_data = CustomDataset(xz_train, y_train, z_train)
    else:
        new_index = datasampling(xz_train, y_train, z_train, our_weights, seed = seed)
        train_data = CustomDataset(xz_train[new_index], y_train[new_index], z_train[new_index])
    
    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = alpha, target_fairness = 'dpeo', knob = 10, replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    
    # ---------------------
    #  Model training
    # ---------------------

    for epoch in range(500):

        tmp_loss = []

        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)

        losses.append(sum(tmp_loss)/len(tmp_loss))
    
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, Unfairness (DP & EO): {}".format(tmp_test['Acc'], max(tmp_test['DP_diff'], tmp_test['EqOdds_diff'])))
    print("----------------------------------------------------------------------")
    

< Seed: 0 >
  Test accuracy: 0.8600000143051147, Unfairness (DP & EO): 0.05686274509803921
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.8500000238418579, Unfairness (DP & EO): 0.0586666666666667
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.8489999771118164, Unfairness (DP & EO): 0.05728571428571422
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.8500000238418579, Unfairness (DP & EO): 0.0586666666666667
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.8489999771118164, Unfairness (DP & EO): 0.05728571428571422
----------------------------------------------------------------------


In [13]:
tmp_acc = []
tmp_unfair = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_unfair.append(max(full_tests[i]['DP_diff'], full_tests[i]['EqOdds_diff']))

print("Test accuracy (avg): {}".format(np.mean(tmp_acc)))
print("Unfairness (DP & EO) (avg): {}".format(np.mean(tmp_unfair)))

Test accuracy (avg): 0.8516000032424926
Unfairness (DP & EO) (avg): 0.057753501400560216
